In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects import numpy2ri
pandas2ri.activate()
import pickle

# Create output directory
output_dir = "./data/processed/expression/autoencoder/"
os.makedirs(output_dir, exist_ok=True)

# Define data paths
data_path = "./data/processed/expression/readcounts_tmm_all/"
metadata_path = "./data/processed/attphe.pkl"

# Load metadata
with open(metadata_path, 'rb') as f:
    metadata = pickle.load(f)

# List all tissue files
tissue_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".pkl")]

# Define Autoencoder using Functional API
def build_autoencoder(input_dim):
    input_layer = keras.Input(shape=(input_dim,))
    x = layers.Dense(10, activation='relu')(input_layer)
    x = layers.Dense(6, activation='relu')(x)
    latent = layers.Dense(3, activation='relu')(x)  # Latent space 
    x = layers.Dense(6, activation='relu')(latent)
    x = layers.Dense(10, activation='relu')(x)
    output = layers.Dense(input_dim, activation='linear')(x)

    autoencoder = keras.Model(inputs=input_layer, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mse')

    # Encoder model to extract latent space
    encoder = keras.Model(inputs=input_layer, outputs=latent)

    return autoencoder, encoder

# Function to process each tissue file
def process_tissue_autoencoder(tissue_file, metadata, min_samples=10):
    tissue_name = os.path.basename(tissue_file).replace(".pkl", "")

    # Load normalized read counts
    with open(tissue_file, 'rb') as f:
        normalized_counts = pickle.load(f)

    sample_ids = normalized_counts.columns
    attr_filtered = metadata[metadata['samp_id'].isin(sample_ids)]

    # Skip tissue if the sample size is smaller than the minimum threshold
    if len(sample_ids) < min_samples:
        print(f"Skipping tissue {tissue_name} due to small sample size ({len(sample_ids)} samples).")
        return

    if normalized_counts.shape[1] != attr_filtered.shape[0]:
        raise ValueError(f"Number of samples in {tissue_name} does not match metadata.")

    # Resave normalized counts as DataFrame (using pandas HDF5 as previously)
    resaved_tissue_file = os.path.join(data_path, f"{tissue_name}_resaved.h5")
    normalized_counts.to_hdf(resaved_tissue_file, key='normalized_counts')
    print(f"Resaved normalized counts for {tissue_name} to {resaved_tissue_file}")

    
    input_dim = normalized_counts.shape[1]
    scaler = StandardScaler()
    normalized_counts_scaled  = scaler.fit_transform(normalized_counts.T).T  # Normalize across samples

    autoencoder, encoder = build_autoencoder(input_dim)

    # Train the autoencoder
    autoencoder.fit(normalized_counts_scaled,  normalized_counts_scaled, epochs=50, batch_size=32)

    # Encode latent features
    latent_features = encoder.predict(normalized_counts_scaled)

    # Load limma package in R
    ro.r('library(limma)')
    
    numpy2ri.activate()
    
    # Transpose expression matrix so rows = samples, columns = genes
    counts_T = normalized_counts.T  # shape: (samples, genes)
    r_counts = ro.conversion.py2rpy(counts_T.to_numpy())  # 
    # Convert latent features
    r_covariates = ro.conversion.py2rpy(np.array(latent_features))

    print("Expression shape:", normalized_counts.shape)
    print("Latent shape:", latent_features.shape)

    # Call R function
    adjusted_expression_data = ro.r['removeBatchEffect'](r_counts, covariates=r_covariates)

    # Convert adjusted data back to pandas DataFrame and transpose it to original shape
    adjusted_expression_df = pd.DataFrame(np.array(adjusted_expression_data),
                                          index=counts_T.index,
                                          columns=counts_T.columns).T  # 
    
    # Save adjusted expression data to pickle file
    result_file = os.path.join(output_dir, f"{tissue_name}.pkl")
    with open(result_file, 'wb') as f:
        pickle.dump(adjusted_expression_df, f)

    print(f"Processed tissue: {tissue_name}")
    print(f"Dimensions of adjusted data: {adjusted_expression_df.shape}")

# Loop through each tissue file and process them
for tissue_file in tissue_files:
    process_tissue_autoencoder(tissue_file, metadata)


Resaved normalized counts for Brain-Hypothalamus to ./data/processed/expression/readcounts_tmm_all/Brain-Hypothalamus_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0179
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1.0112
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1.0064
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1.0028
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9999
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9975
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9956
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9944
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9932
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.9920
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9908
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9895
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.9883
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 555 probe(s) 



Expression shape: (14, 202)
Latent shape: (14, 3)
Processed tissue: Brain-Hypothalamus
Dimensions of adjusted data: (14, 202)
Resaved normalized counts for Muscle-Skeletal to ./data/processed/expression/readcounts_tmm_all/Muscle-Skeletal_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0008
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9990
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9974
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9957
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9938
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.9916
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.9888
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.9855
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9819
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.9779
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9734
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/s

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 162 probe(s) 



Expression shape: (14, 174)
Latent shape: (14, 3)
Processed tissue: Cells-EBV-transformed lymphocytes
Dimensions of adjusted data: (14, 174)
Resaved normalized counts for Pituitary to ./data/processed/expression/readcounts_tmm_all/Pituitary_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.9998
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.9986
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.9977
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9969
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.9961
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.9952
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9943
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9933
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9922
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.9911
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9897
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms

...

Expression shape: (14, 205)
Latent shape: (14, 3)
Coefficients not estimable: 2 
Processed tissue: Brain-Putamen (basal ganglia)
Dimensions of adjusted data: (14, 205)
Resaved normalized counts for Adrenal Gland to ./data/processed/expression/readcounts_tmm_all/Adrenal Gland_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0013
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.9996
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.9980
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9966
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9950
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.9934
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.9915
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.9896
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9876
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.9853
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.9828
Epoch 12

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 205 probe(s) 



Expression shape: (14, 258)
Latent shape: (14, 3)
Coefficients not estimable: 2 3 
Processed tissue: Adrenal Gland
Dimensions of adjusted data: (14, 258)
Skipping tissue Fallopian Tube due to small sample size (9 samples).
Resaved normalized counts for Brain-Cortex to ./data/processed/expression/readcounts_tmm_all/Brain-Cortex_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0095
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 1.0049
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 1.0016
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.9990
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.9971
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.9957
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.9944
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.9931
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.9917
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.9903
Epoch 11/50
1/1 ━━━━━

R[write to console]: Zusätzlich: 
R[write to console]: Warnmeldung:

R[write to console]: Partial NA coefficients for 258 probe(s) 



Expression shape: (14, 255)
Latent shape: (14, 3)
Processed tissue: Brain-Cortex
Dimensions of adjusted data: (14, 255)
Resaved normalized counts for Esophagus-Mucosa to ./data/processed/expression/readcounts_tmm_all/Esophagus-Mucosa_resaved.h5
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 1.0006
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.9995
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.9987
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.9979
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.9969
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.9960
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.9948
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9934
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.9920
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.9903
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.9883
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step